# Example: evolving a buggy_coder into a github asana syncing bot

## Prerequisite

- OPENAI_API_KEY
- E2B_API_KEY
- COMPOSIO_API_KEY
- ASANA_WORKSPACE_ID
- ASANA_TEAM_GID

In [1]:
import os
import sys
import json
repo_path = os.path.abspath("..")  # add parent path for imports
from langgraph.checkpoint.memory import MemorySaver

# 1. Initialize your checkpointer
memory = MemorySaver()

from agents.eval_agent.graph import build_graph

[2025-12-18 01:28:15,938][WARNING] Failed to load LangChain docs tools: asyncio.run() cannot be called from a running event loop
/Users/lokesh/Desktop/seer/Code/seer/shared/tools/mcp_client.py:34: RuntimeWarning: coroutine 'MultiServerMCPClient.get_tools' was never awaited
  LANGCHAIN_DOCS_TOOLS = []


2025-12-18 01:28:16 | eval_agent.graph | WARNING | DATABASE_URI not set. Human-in-the-loop interrupts will not work.
2025-12-18 01:28:16 | eval_agent.graph | WARNING | Set DATABASE_URI environment variable to enable interrupts.


In [3]:
graph = build_graph()
memory = MemorySaver()
eval_agent = graph.compile(checkpointer=memory)

## Step 1: Agent Expectation alignment

In [4]:
GITHUB_ASANA_BOT_EXPECTATIONS = """
Evaluate my agent buggy_coder
The agent should sync Asana ticket updates when a GitHub PR is merged on it's own. Whenever i merge a PR it should search for realted asana tickets and update/close them.
"""

BUGGY_CODER_REPO_SLUG = "seer-engg/langgraph-skeleton"

USER_ID = "lokesh@getseer.dev"

In [5]:
compiled_inputs = {
    "messages": [
        {
            "type": "human",
            "content": GITHUB_ASANA_BOT_EXPECTATIONS
        }
    ],
    "step": "alignment",
    "input_context": {
        "integrations": {
            "github": {
                "name": BUGGY_CODER_REPO_SLUG,
            }
        },
        "user_id": USER_ID
    }
}

In [6]:
import uuid
thread_id = str(uuid.uuid4())
from langchain_core.runnables import RunnableConfig

results = await eval_agent.ainvoke(compiled_inputs, config=RunnableConfig(configurable={"thread_id": thread_id}))

2025-12-18 01:28:32 | eval_agent.plan | INFO | Resolved MCP services (requested=['asana', 'github']): ['asana', 'github']


In [7]:
print("agent name: ", results.get('context').agent_name)
print("mcp services required: ", results.get('context').mcp_services)
print("functional requirements: ", "\n".join(results.get('context').functional_requirements))

agent name:  buggy_coder
mcp services required:  ['asana', 'github']
functional requirements:  Monitor the specified GitHub repository and detect when a pull request is merged into the main branch and trigger processing automatically (near‑real‑time).
For each merged PR, search the PR title, body, and all associated commits for references to Asana tasks or Asana task URLs and identify all referenced task IDs (supporting multiple tasks).
If no Asana tasks are found for a merged PR, record that fact and make no changes to Asana tasks.
For every identified Asana task referenced by the PR, add a comment to the Asana task that includes the PR URL, merge commit SHA, merger identity, and a concise summary that the PR was merged and which repository/branch it merged into.
If the PR text or commits include an explicit closing intent for a referenced Asana task (for example, common close keywords used immediately with the task reference), mark that Asana task complete/closed; otherwise leave the

## Step 2: Eval Generation

In [8]:
eval_gen_inputs = {
    "step": "plan",
}

In [9]:
planning_results = await eval_agent.ainvoke(eval_gen_inputs, config=RunnableConfig(configurable={"thread_id": thread_id}))

2025-12-18 01:29:36 | eval_agent.plan.get_reflections | INFO | get_reflections: No Neo4j memory available, returning empty
2025-12-18 01:29:36 | eval_agent.plan.generate_evals | INFO | Using 'agentic' (structured output) test generation.
2025-12-18 01:29:36 | eval_agent.plan.generate_evals | INFO | Using reasoning_effort: medium
2025-12-18 01:30:56 | eval_agent.plan.generate_evals | INFO | plan.generate: produced 1 tests (agent=buggy_coder)
2025-12-18 01:31:31 | eval_agent.plan.filter_tools | INFO | Selected 47 tools: ['ASANA_ADD_FOLLOWERS_TO_TASK', 'ASANA_ADD_TASK_TO_SECTION', 'ASANA_CREATE_A_TASK', 'ASANA_CREATE_TASK_COMMENT', 'ASANA_DUPLICATE_PROJECT', 'ASANA_DUPLICATE_TASK', 'ASANA_GET_A_TASK', 'ASANA_GET_CURRENT_USER', 'ASANA_GET_MULTIPLE_PROJECTS', 'ASANA_GET_MULTIPLE_TASKS', 'ASANA_GET_PROJECTS_FOR_TEAM', 'ASANA_GET_STORIES_FOR_TASK', 'ASANA_GET_STORY', 'ASANA_GET_TASKS_FROM_A_PROJECT', 'ASANA_GET_TYPEAHEAD_OBJECTS', 'ASANA_GET_USERS_FOR_WORKSPACE', 'ASANA_GET_WORKSPACE', 'ASANA

In [10]:
for example in planning_results.get('dataset_examples'):
    print(example.to_markdown())
## Step 2: Test Execution

### Dataset Example `89de4973-8d8c-4c83-b644-19147bb3958a`

- **Status**: `active`

#### Input Message

```
{
  "event": "pull_request",
  "action": "closed",
  "pull_request": {
    "title": "Merge: Fix login bug and Refactor auth module - ASYNCTEST-001",
    "body": "This PR fixes the login bug.\n\nFixes Asana task: Fix login bug - ASYNCTEST-001\nAlso references Asana task: Refactor auth module - ASYNCTEST-001\n",
    "merged": true,
    "merged_by": { "login": "ci-bot" },
    "merge_commit_sha": "merge-sha-async-001",
    "html_url": "https://github.com/seer-engg/label-edgecase-repo/pull/asynctest-001"
  },
  "repository": { "full_name": "seer-engg/label-edgecase-repo" }
}
```

#### Expected Output

- **Expected action**: On receiving a PR merged event, find Asana tasks referenced in the PR (by name mentions in the PR body/title), mark any referenced open Asana tasks as completed, and add a comment on each closed-by-this-PR Asana task containing the PR title and a link or the merge 

## Step 3: Testing

In [11]:
testing_inputs = {
    "step": "testing",
}
testing_results = await eval_agent.ainvoke(testing_inputs, config=RunnableConfig(configurable={"thread_id": thread_id}))

2025-12-18 01:31:31 | eval_agent.run | INFO | run.prepare: dataset=seer_eval_20251218-0131 experiment=seer-eval-local-20251218-013131
2025-12-18 01:31:31 | eval_agent.plan | INFO | plan.provision: provisioning sandbox (repo=https://github.com/seer-engg/langgraph-skeleton branch=main)
2025-12-18 01:31:31 | sandbox.initialize | INFO | Creating E2B sandbox for codex...
2025-12-18 01:31:33 | sandbox.initialize | INFO | Sandbox created: itro5x8j6phhprsekr3er
2025-12-18 01:31:33 | sandbox.initialize | INFO | Cloning/updating repository inside sandbox (shell)...
2025-12-18 01:32:27 | sandbox.initialize | INFO | Setup project: Obtaining file:///home/user/langgraph-skeleton
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build 

ValueError: LANGFUSE_SECRET_KEY environment variable is required for experiment upload.